# Function Call

## Demo code to make function calls
A quick demo code to make function calls. 

Please ensure you have *.env* file in your HOME/Documents/src/openai/ folder

In [ ]:
#!pip install pysqlite3

## Initialize the Environment

In [ ]:
from openai import AzureOpenAI
from dotenv import load_dotenv
from pprint import pprint
import pandas as pd
import json
import os

env_path = os.getenv("HOME") + "/Documents/src/openai/.env"
load_dotenv(dotenv_path=env_path, verbose=True)

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://pvg-azure-openai-uk-south.openai.azure.com"

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2023-05-15"
)

In [ ]:
import requests
df = pd.read_csv("data/employees.csv")

def get_colleague_info(colleague_name: str):
    df1 = df[df.name == colleague_name.lower()]
    if df1.empty:
        return None
    else:
        colleague_info = {
            "name": df1.name.loc[df1.index[0]],
            "role": df1.role.loc[df1.index[0]]
        }
        return json.dumps(colleague_info)
    

amap_key = os.getenv("AMAP_KEY")

df2 = pd.read_csv("data/base.csv")

def get_weather_by_colleague(colleague_name: str):
    df3 = df2[df2.name == colleague_name.lower()]
    if df3.empty:
        return None
    else:
        city_id = df3.city_id.loc[df3.index[0]]
        
    url = f"https://restapi.amap.com/v3/weather/weatherInfo?key={amap_key}&city={city_id}"
    r = requests.get(url)
    result = r.json()
    if "lives" in result and result["lives"]:
        return result["lives"][0]["weather"]
    return None

In [ ]:
tools = [
    {"type": "function", 
         "function": {
            "name": "get_colleague_info", 
            "description": "Get colleagues' information",
            "parameters": {
                "type":"object",
                "properties": {
                    "colleague_name": {
                        "type": "string", 
                        "description": "the name of the colleague"
                    }
                }
            }
        }
    },
    {"type": "function", 
         "function": {
            "name": "get_weather_by_colleague", 
            "description": "Get weather by colleague's name",
            "parameters": {
                "type":"object",
                "properties": {
                    "colleague_name": {
                        "type": "string", 
                        "description": "the name of the colleague"
                    }
                }
            }
        }
    }
]

## Run a Single Function Calling

In [ ]:
system_prompt = "You are a helpful assistant."
user_prompt = "what's the role of my colleague Michael?"
messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
           ]

response = client.chat.completions.create(
    model="gpt-35-turbo", 
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    tools=tools, 
    tool_choice = 'auto'
)

response_message = response.choices[0].message

if response.choices[0].finish_reason == "tool_calls":
    print("GPT asked us to call a function.")

    messages.append(response_message)

    for tool_call in response.choices[0].message.tool_calls: 
        function_name = tool_call.function.name
        params = json.loads(tool_call.function.arguments)
    
        function_response = get_colleague_info (
            **params
        )

        messages.append({"role": "tool", "tool_call_id": tool_call.id, "name": function_name, "content": function_response})
        
    second_response = client.chat.completions.create(
        model="gpt-35-turbo", 
        messages = messages,
    )
        
    print(second_response.choices[0].message.content)

## Run Multiple Function Callings

In [ ]:
system_prompt = "You are a helpful assistant."
user_prompt = "what's the role and weather of my colleague Michael, Hunter and Mark?"
messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
           ]

response = client.chat.completions.create(
    model="gpt-35-turbo", 
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    tools=tools, 
    tool_choice = 'auto'
)

response_message = response.choices[0].message

if response.choices[0].finish_reason == "tool_calls":
    print("GPT asked us to call a function.")

    messages.append(response_message)

    for tool_call in response.choices[0].message.tool_calls: 
        function_name = tool_call.function.name
        params = json.loads(tool_call.function.arguments)

        if function_name == "get_colleague_info":
            function_response = get_colleague_info (
                **params
            )
        else:
            function_response = get_weather_by_colleague (
                **params
            )

        messages.append({"role": "tool", "tool_call_id": tool_call.id, "name": function_name, "content": function_response})

    second_response = client.chat.completions.create(
        model="gpt-35-turbo", 
        messages = messages,
    )
        
    print(second_response.choices[0].message.content)

## Define LangChain Tools

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def get_colleague_info_enh(colleague_name: str):

    """Get the information of a colleague.

    Args:
        colleague_name: name of the colleague
    """
    
    df1 = df[df.name == colleague_name.lower()]
    if df1.empty:
        return None
    else:
        colleague_info = {
            "name": df1.name.loc[df1.index[0]],
            "role": df1.role.loc[df1.index[0]]
        }
        return json.dumps(colleague_info)

@tool
def get_weather_by_colleague_enh(colleague_name: str):

    """Get the weather of a colleague.

    Args:
        colleague_name: name of the colleague
    """
    
    df3 = df2[df2.name == colleague_name.lower()]
    if df3.empty:
        return None
    else:
        city_id = df3.city_id.loc[df3.index[0]]
        
    url = f"https://restapi.amap.com/v3/weather/weatherInfo?key={amap_key}&city={city_id}"
    r = requests.get(url)
    result = r.json()
    if "lives" in result and result["lives"]:
        return result["lives"][0]["weather"]
    return None

print(json.dumps(convert_to_openai_tool(get_colleague_info_enh), indent=2))

## Run LangChain Agents

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain.agents import Tool, AgentExecutor, create_openai_tools_agent
from langchain import hub
 
model = AzureChatOpenAI(deployment_name="gpt-35-turbo")

tools = [
    Tool(
        name = "get_colleague_info_enh",
        func = get_colleague_info_enh.run,
        description = "useful for colleague information extraction"
    ),
    Tool(
        name = "get_weather_by_colleague_enh",
        func = get_weather_by_colleague_enh.run,
        description = "useful for colleague weather extraction"
    )
]

prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_openai_tools_agent(model, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=False, handle_parsing_errors=True
)
agent_executor.invoke({"input": "what's the role and weather of my colleague Michael, Hunter and Jack?"})

## Query SQL

In [ ]:
def get_sql_completion(messages, model="gpt-35-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
        tools=[{  # 摘自 OpenAI 官方示例 https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_with_chat_models.ipynb
            "type": "function",
            "function": {
                "name": "ask_database",
                "description": "Use this function to answer user questions about business. \
                            Output should be a fully formed SQL query.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": f"""
                            SQL query extracting info to answer the user's question.
                            SQL should be written using this database schema:
                            {database_schema_string}
                            The query should be returned in plain text, not in JSON.
                            The query should only contain grammars supported by SQLite.
                            """,
                        }
                    },
                    "required": ["query"],
                }
            }
        }],
    )
    return response.choices[0].message

In [ ]:
database_schema_string = """
CREATE TABLE orders (
    id INT PRIMARY KEY NOT NULL, -- Key field, should not be null
    customer_id INT NOT NULL, -- customer id field, should not be null either
    product_id STR NOT NULL, -- product id field
    price DECIMAL(10,2) NOT NULL, -- price of the order
    status INT NOT NULL, -- status of the order. 0 means not yet paid，1 means paid，2 means already refunded
    create_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP, -- creation date, by default it's right now
    pay_time TIMESTAMP -- pay date, could be empty
);
"""

In [ ]:
import sqlite3

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

cursor.execute(database_schema_string)

mock_data = [
    (1, 1001, 'TAPESTRY_BAG_1', 50.00, 0, '2024-2-10 10:00:00', None),
    (2, 1001, 'TAPESTRY_BAG_2', 75.50, 1, '2024-2-16 11:00:00', '2024-02-16 12:00:00'),
    (3, 1002, 'ASICS_SHOE_1', 85.25, 1, '2024-2-17 12:30:00', '2024-02-17 13:00:00'),
    (4, 1002, 'HANAHAUSE_COKE', 60.75, 1, '2024-2-20 14:00:00', '2024-02-20 15:00:00'),
    (5, 1003, 'SMALL_POTATO', 5.00, 0, '2024-2-28 16:00:00', None)
]

for record in mock_data:
    cursor.execute('''
    INSERT INTO orders (id, customer_id, product_id, price, status, create_time, pay_time)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', record)

conn.commit()

In [ ]:
def ask_database(query):
    cursor.execute(query)
    records = cursor.fetchall()
    return records


#prompt = "which customer spent the most accumulately? How much did he or she spend in total?"
prompt = "what's the total sales of each product?"

messages = [
    {"role": "system", "content": "please answer the question based on the order table"},
    {"role": "user", "content": prompt}
]
response = get_sql_completion(messages)
if response.content is None:
    response.content = ""
messages.append(response)
print(response)

if response.tool_calls is not None:
    tool_call = response.tool_calls[0]
    if tool_call.function.name == "ask_database":
        arguments = tool_call.function.arguments
        args = json.loads(arguments)
        print(args["query"])
        result = ask_database(args["query"])
        print(result)

        messages.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": "ask_database",
            "content": str(result)
        })
        response = get_sql_completion(messages)
        print(response.content)